In [1]:
import pandas as pd
pd.options.display.max_rows=100
import numpy as np
import requests

In [32]:
coord = pd.read_csv("coordinates.csv")
coord

,BranchID,Location,Latitude,Longitude
0,IIT001,IIT Area,19.134344,72.890891
1,ANE002,Andheri East,19.114737,72.850938
2,BRV003,Borivali,19.231771,72.828090
3,MUU004,University Area,19.102123,72.804951
4,BKC005,BKC Region,19.063391,72.851835
5,CHM006,Chembur East,19.020779,72.834109
6,CLB007,Colaba,18.922700,72.830696


In [38]:
cat_score = pd.read_csv("category_scores.csv")
cat_score

,Category,Score
0,Chinese Restaurant,100
1,Indian Restaurant,90
2,Coffee Shop,100
3,Pizza Place,120
4,Restaurant (Others),80
5,Ice Cream Shop,80
6,Bar,100
7,Hotel,100
8,Others,50


In [3]:
CLIENT_ID = 'DS0UKDSWSCFUFY0TV1YPIDYIEGUWQCSKZGHOIP45FLGZDC1X' # your Foursquare ID
CLIENT_SECRET = 'VYX0TO1JCGIJ4O3TSM3PW41M3XBUKGIESGRRJNZYYRXGHGZ4' # your Foursquare Secret
VERSION = '20181223' # Foursquare API version

In [ ]:
latitude_test = 19.0#28.4228855
longitude_test = 72.8#76.8493513
branch_id_test = 'TST000'
location_test = 'Mumbai Test Location'

LIMIT = 100
radius = 5000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude_test, 
    longitude_test, 
    VERSION, 
    radius, 
    LIMIT)

In [ ]:
results = requests.get(url).json()
testresults=results["response"]['groups'][0]['items']

In [ ]:
venues_list_test = []
branch_id = branch_id_test
location = location_test
latitude = latitude_test
longitude = longitude_test
            
venues_list_test.append([(
            branch_id,
            location, 
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in testresults])
nearby_venues_test = pd.DataFrame([item for venue_list in venues_list_test for item in venue_list])
nearby_venues_test.columns = ['Branch ID',
            'Location', 
            'Location Latitude', 
            'Location Longitude', 
            'Venue', 
            'Venue Latitude', 
            'Venue Longitude', 
            'Venue Category']
nearby_venues_test

In [22]:
def getNearbyVenues(branchid, names, latitudes, longitudes, radius=1000, LIMIT=100):
    print('Location queried:')
    venues_list=[]
    for branch_idf, locationf, latitudef, longitudef in zip(branchid, names, latitudes, longitudes):
        print(locationf)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudef, 
            longitudef, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue 
        venues_list.append([(branch_idf,
                locationf, 
                latitudef, 
                longitudef, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['BranchID',
                    'Location', 
                    'Location Latitude', 
                    'Location Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
    
    return(nearby_venues)

In [23]:
mumbai_venues = getNearbyVenues(branchid=coord['BranchID'],
                                    names=coord['Location'],
                                    latitudes=coord['Latitude'],
                                    longitudes=coord['Longitude']
                                  )

Location queried:
IIT Area
Andheri East
Borivali
University Area
BKC Region
Chembur East
Colaba


In [24]:
backup=mumbai_venues
print('Shape: ',mumbai_venues.shape)
mumbai_venues.head()

Shape:  (250, 8)


,BranchID,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,IIT001,IIT Area,19.134344,72.890891,Skky,19.135841,72.898494,Restaurant
1,IIT001,IIT Area,19.134344,72.890891,The Residence Hotel And Convention Center @ Powai,19.134862,72.898510,Hotel
2,IIT001,IIT Area,19.134344,72.890891,Hotel Ramada,19.136170,72.898967,Hotel
3,IIT001,IIT Area,19.134344,72.890891,L&T Canteen LR2,19.129449,72.896191,Food Court
4,IIT001,IIT Area,19.134344,72.890891,Cafe Ambrosia,19.135795,72.898502,Restaurant


In [13]:
mumbai_venues.loc[mumbai_venues['BranchID']=='ANE002'].head()

,BranchID,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
100,ANE002,Andheri East,19.114737,72.850938,Merwans Cake shop,19.119300,72.845418,Bakery
101,ANE002,Andheri East,19.114737,72.850938,Naturals,19.111204,72.837255,Ice Cream Shop
102,ANE002,Andheri East,19.114737,72.850938,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
103,ANE002,Andheri East,19.114737,72.850938,Babu Cha Vada,19.103313,72.848576,Snack Place
104,ANE002,Andheri East,19.114737,72.850938,Anand Dosa,19.102982,72.836861,Breakfast Spot


In [14]:
mumbai_venues.loc[mumbai_venues['BranchID']=='BRV003'].head()

,BranchID,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
200,BRV003,Borivali,19.231771,72.82809,Water Kingdom,19.232115,72.821438,Water Park
201,BRV003,Borivali,19.231771,72.82809,Kabab Chini,19.232559,72.827470,Restaurant
202,BRV003,Borivali,19.231771,72.82809,Green channel,19.232521,72.828032,Fast Food Restaurant
203,BRV003,Borivali,19.231771,72.82809,Tight House,19.232452,72.828171,Pub
204,BRV003,Borivali,19.231771,72.82809,Water Kingdom,19.231699,72.807381,Water Park


In [25]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['BranchID'] = mumbai_venues['BranchID'] 

# move neighborhood column to the first column
bid=mumbai_onehot['BranchID']
mumbai_onehot.drop(['BranchID'], axis=1, inplace=True)
mumbai_onehot.insert(0, 'BranchID', bid)

print('Shape:',mumbai_onehot.shape)
mumbai_onehot.head()

Shape: (250, 90)


,BranchID,Accessories Store,Airport Terminal,Arcade,Art Gallery,Asian Restaurant,Auto Workshop,Bakery,Bar,Beach,...,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Track,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Shop
0,IIT001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,IIT001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,IIT001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,IIT001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,IIT001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
mumbai_grouped = mumbai_onehot.groupby('BranchID').mean().reset_index()
print('Shape: ', mumbai_grouped.shape)
mumbai_grouped

Shape:  (7, 90)


,BranchID,Accessories Store,Airport Terminal,Arcade,Art Gallery,Asian Restaurant,Auto Workshop,Bakery,Bar,Beach,...,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Track,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Shop
0,ANE002,0.0,0.000000,0.000000,0.00,0.00,0.027778,0.027778,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.027778
1,BKC005,0.0,0.043478,0.000000,0.00,0.00,0.000000,0.000000,0.043478,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,BRV003,0.0,0.000000,0.055556,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.055556,0.00,0.000000,0.000000,0.000000,0.055556,0.000000
3,CHM006,0.0,0.000000,0.016129,0.00,0.00,0.000000,0.032258,0.048387,0.016129,...,0.00,0.016129,0.016129,0.000000,0.00,0.016129,0.016129,0.016129,0.000000,0.000000
4,CLB007,0.0,0.000000,0.000000,0.01,0.02,0.000000,0.020000,0.050000,0.000000,...,0.01,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.020000,0.000000,0.000000
5,IIT001,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,MUU004,1.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['BranchID']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
location_venues_sorted = pd.DataFrame(columns=columns)
location_venues_sorted['BranchID'] = mumbai_grouped['BranchID']

for ind in np.arange(mumbai_grouped.shape[0]):
    location_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

location_venues_sorted

,BranchID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,ANE002,Indian Restaurant,Vegetarian / Vegan Restaurant,Café
1,BKC005,Indian Restaurant,Pizza Place,Café
2,BRV003,Restaurant,Café,Playground
3,CHM006,Indian Restaurant,Ice Cream Shop,Chinese Restaurant
4,CLB007,Indian Restaurant,Café,Bar
5,IIT001,Restaurant,Hotel,Indian Restaurant
6,MUU004,Accessories Store,Halal Restaurant,Deli / Bodega


In [46]:
Score=[270,310,230,270,290,270,180]

In [47]:
location_venues_sorted['Score']=Score

In [49]:
branch_indices = coord.join(location_venues_sorted.set_index('BranchID'), on='BranchID')
branch_indices

,BranchID,Location,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Score
0,IIT001,IIT Area,19.134344,72.890891,Restaurant,Hotel,Indian Restaurant,270
1,ANE002,Andheri East,19.114737,72.850938,Indian Restaurant,Vegetarian / Vegan Restaurant,Café,270
2,BRV003,Borivali,19.231771,72.828090,Restaurant,Café,Playground,230
3,MUU004,University Area,19.102123,72.804951,Accessories Store,Halal Restaurant,Deli / Bodega,180
4,BKC005,BKC Region,19.063391,72.851835,Indian Restaurant,Pizza Place,Café,310
5,CHM006,Chembur East,19.020779,72.834109,Indian Restaurant,Ice Cream Shop,Chinese Restaurant,270
6,CLB007,Colaba,18.922700,72.830696,Indian Restaurant,Café,Bar,290
